In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [4]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

RowNumber  CustomerId   Surname  ...  IsActiveMember EstimatedSalary Exited
0          1    15634602  Hargrave  ...               1       101348.88      1
1          2    15647311      Hill  ...               1       112542.58      0
2          3    15619304      Onio  ...               0       113931.57      1
3          4    15701354      Boni  ...               0        93826.63      0
4          5    15737888  Mitchell  ...               1        79084.10      0

[5 rows x 14 columns]

In [5]:
#Preprocess the data
#Drop irrelevant columns

data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

CreditScore Geography  Gender  ...  IsActiveMember  EstimatedSalary  Exited
0             619    France  Female  ...               1        101348.88       1
1             608     Spain  Female  ...               1        112542.58       0
2             502    France  Female  ...               0        113931.57       1
3             699    France  Female  ...               0         93826.63       0
4             850     Spain  Female  ...               1         79084.10       0
...           ...       ...     ...  ...             ...              ...     ...
9995          771    France    Male  ...               0         96270.64       0
9996          516    France    Male  ...               1        101699.77       0
9997          709    France  Female  ...               1         42085.58       1
9998          772   Germany    Male  ...               0         92888.52       1
9999          792    France  Female  ...               0         38190.78       0

[10000 rows x 11 columns]

In [6]:
#Encode Categorical Variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

CreditScore Geography  Gender  ...  IsActiveMember  EstimatedSalary  Exited
0             619    France       0  ...               1        101348.88       1
1             608     Spain       0  ...               1        112542.58       0
2             502    France       0  ...               0        113931.57       1
3             699    France       0  ...               0         93826.63       0
4             850     Spain       0  ...               1         79084.10       0
...           ...       ...     ...  ...             ...              ...     ...
9995          771    France       1  ...               0         96270.64       0
9996          516    France       1  ...               1        101699.77       0
9997          709    France       0  ...               1         42085.58       1
9998          772   Germany       1  ...               0         92888.52       1
9999          792    France       0  ...               0         38190.78       0

[10000 rows x 11 columns]

In [7]:
#OneHotEncode Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [9]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]

In [10]:
#combine one hot encoder columns with original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data

CreditScore  Gender  ...  Geography_Germany  Geography_Spain
0             619       0  ...                0.0              0.0
1             608       0  ...                0.0              1.0
2             502       0  ...                0.0              0.0
3             699       0  ...                0.0              0.0
4             850       0  ...                0.0              1.0
...           ...     ...  ...                ...              ...
9995          771       1  ...                0.0              0.0
9996          516       1  ...                0.0              0.0
9997          709       0  ...                0.0              0.0
9998          772       1  ...                1.0              0.0
9999          792       0  ...                0.0              0.0

[10000 rows x 13 columns]

In [11]:
#Save the encoder and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)




In [12]:
#Divide the dataset into dependent and independent features
x=data.drop('Exited',axis=1)
y=data['Exited']

#Split the data in training and testing sets
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

#Scale these features
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)


In [13]:
x_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [14]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [15]:
data

CreditScore  Gender  ...  Geography_Germany  Geography_Spain
0             619       0  ...                0.0              0.0
1             608       0  ...                0.0              1.0
2             502       0  ...                0.0              0.0
3             699       0  ...                0.0              0.0
4             850       0  ...                0.0              1.0
...           ...     ...  ...                ...              ...
9995          771       1  ...                0.0              0.0
9996          516       1  ...                0.0              0.0
9997          709       0  ...                0.0              0.0
9998          772       1  ...                1.0              0.0
9999          792       0  ...                0.0              0.0

[10000 rows x 13 columns]

ANN IMPLEMENTATION


In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [17]:
#Building our ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),#HL1 connected with input layer
    Dense(32,activation='relu'), #HL2
    Dense(1,activation='sigmoid') #output layer


])

In [18]:
x_train.shape[1]

12

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss


In [25]:
#compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [26]:
#Set up the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [30]:
#set up earlystopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [31]:
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]

)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3407 - accuracy: 0.8616 - val_loss: 0.3493 - val_accuracy: 0.8525
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3407 - accuracy: 0.8630 - val_loss: 0.3428 - val_accuracy: 0.8580
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3357 - accuracy: 0.8630 - val_loss: 0.3471 - val_accuracy: 0.8610
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3352 - accuracy: 0.8621 - val_loss: 0.3552 - val_accuracy: 0.8550
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3291 - accuracy: 0.8660 - val_loss: 0.3435 - val_accuracy: 0.8580
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3317 - accuracy: 0.8633 - val_loss: 0.3464 - val_accuracy: 0.8565
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3286 - accuracy: 0.8648 - val_loss: 0.3423 - val_accuracy: 0.8615

In [32]:
model.save('model.h5')

d:\GenAI\myenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
#Load Tensorboard Extension
%load_ext tensorboard

In [36]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 1220), started 0:05:31 ago. (Use '!kill 1220' to kill it.)

In [ ]:
#Load the pickle file
